# Tutorial \#4: Variational Algorithms with LibKet

In this fourth part of the tutorial you will learn to implement variational algorithms with LibKet.

## Getting started

Let's include **LibKet**'s main headerfile and import its namespaces

In [ ]:
#include "LibKet.hpp"
using namespace LibKet;
using namespace LibKet::circuits;
using namespace LibKet::filters;
using namespace LibKet::gates;

## Parameterized rotation gates

The three rotation gates `rx(theta)`, `ry(theta)`, and `rz(theta)` are defined mathematically as follows

$$
R_x(\theta) 
= 
\begin{pmatrix}
\phantom{-i} \cos(\theta/2) & -i \sin(\theta/2)\\
-i \sin(\theta/2) & \phantom{-i} \cos(\theta/2)
\end{pmatrix},
\quad
R_y(\theta) 
=
\begin{pmatrix}
\cos(\theta/2) & -\sin(\theta/2)\\
\sin(\theta/2) & \phantom{-}\cos(\theta/2)
\end{pmatrix},
\quad
R_z(\theta) 
= 
\begin{pmatrix}
\exp(-i\theta/2) & 0\\
0 & \exp(i\theta/2)
\end{pmatrix}.
$$

With their help, the Hadamard gate $ H = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1\end{pmatrix} $ can be written equivalently in terms of two rotation gates, namely

$$
\begin{align*}
H & \mapsto R_x(\pi)R_y(\pi/2) && (\text{Rule 1}),\\
H & \mapsto R_y(-\pi/2)R_x(\pi) && (\text{Rule 2}),\\
H & \mapsto R_y(\pi/2)R_z(\pi) && (\text{Rule 3}),\\
H & \mapsto R_z(\pi)R_y(-\pi/2) && (\text{Rule 4}).
\end{align*}
$$


#### Exercise 1
Check that the first substitution rule yields the same state (up to multiplication with the imaginary number $i$) when applied to the state $\lvert 0 \rangle$. 

To do so, write a quantum expression (*without measurement!*) and execute it with the QuEST simulator. **LibKet** provides predefined constants `QConst_M_PI` and `QConst_M_PI_2` for the values $\pi$ and $\pi/2$, respectively. If you want to test other rotation angles use `QConst(value)` where `value` is the value of the rotation angle. The interface of **LibKet**'s rotation gates reads `rx(QConst_t, expr)` and the same for `ry` and `rz`.

In [ ]:
auto expr1 = rx(QCONST_M_PI,ry(QConst_M_PI_2,init()));

// alternatively
// auto expr1 = rx(QConst(3.14159265358979323846),ry(QConst(1.57079632679489661923),init()));

QDevice<QDeviceType::quest, 1> device1;
device1(expr1);
device1.eval(1);

std::cout << device1.reg() << std::endl;

## Variable parameters
So far, the parameters that we have passed to the rotation gates have been constants, e.g. `QConst(3.14159265358979323846)`. 

Some quantum backends support *variable* parameters that can be varied between multiple executions of the quantum computation without the need to recompile the quantum circuit. Note that this feature is still experimental in **LibKet** and that it might not work for all quantum backends.

The first step consists in defining one or more `QVar_t<id>` objects with a unique `id` for each variable. 

Variables can be assigned an initial value during their constructions
```cpp
QVar_t<0> var0(3.14159265358979323846);
```
and updated at any place in the program
```cpp
var0 = 1.57079632679489661923;
```

The second step consists in formulating a quantum expression with the variable object in lieu of the `QConst` object
```cpp
auto expr = rx(var0, ry(var1, init()));
```

This quantum expression needs to be loaded into the quantum device *once* 
```cpp
QDevice<QDeviceType::quest, 1> device;
device(expr);
```
and can be reevaluated many times
```cpp
device.eval(1);
std::cout << device.reg() << std::endl;

var0 = 3.14159265358979323846;
device.eval(1);
std::cout << device.reg() << std::endl;

var0 = 1.57079632679489661923;
device.eval(1);
std::cout << device.reg() << std::endl;
```

#### Exercise 2
Write a small quantum program that evaluates the expression $R_x(\theta_0)R_y(\theta_1)$ for $\theta_0,\theta_1\in[0,\pi]$. 

We suggest that you implement the variables, the quantum expression and its loading into the quantum device in the first code block and implement the grid search (double for loop) in the second code block.

In [ ]:
// Your definition of variables, the quantum expression, and the quantum device goes here

QVar_t<0> var0(0.0); 
QVar_t<1> var1(0.0);

auto expr2 = rx(var0, ry(var1, init()));

QDevice<QDeviceType::quest, 1> device2; device2(expr2);

In [ ]:
// Your grid search goes here

for (int i = 0; i < 11; i++) {
    for (int j = 0; j < 11; j++) {
        var0 = 3.14159265358979323846/10*i;
        var1 = 3.14159265358979323846/10*j;
        
        device.eval(1);
        std::cout << "i=" << i << " j=" << j << "\n"
                  << device.reg() << std::endl;
    }
}

## The Variational Quantum Eigensolver (VQE)

### The Hamiltonian
First, let us define a structute that stores the coefficients $c_0$ and $c_1$ of the Hamiltonian $H=c_0 X\otimes I + c_1 I\otimes X$

In [ ]:
struct HamiltonianCoeffs
{
  double c[2] = {0.5, 0.6};
};

### The VQE structure
Next, we define the structure `VQE` that provides the following functionalities

*   generation of the **1-qubit U3 ansatz** (`U3(theta, phi, lambda`)

*   generation of the extended **2-qubit U3 ansatz** (`genAnsatz(expr, params)`)

*   conversion of history data to expectation values (`histToExp(hist)`)

*   main driver routine (`runVQE(params, h_coeffs, shots)`)

In [ ]:
struct VQE
{
  template<int index, typename Expr>
  auto U3(Expr&& expr, double theta, double phi, double lambda)
  {    
    QVar_t<3*index,1>   var_theta(theta*2*M_PI);
    QVar_t<3*index+1,1> var_phi(phi*2*M_PI);
    QVar_t<3*index+2,1> var_lambda(lambda*2*M_PI);

    return rz(var_phi,
              rx(-QConst_M_PI_2, 
                 rz(var_theta,
                    rx(QConst_M_PI_2, 
                       rz(var_lambda, 
                          expr
                          )
                       )
                    )
                 )
              );
  }

  template<typename Expr, typename Params>
  auto genAnsatz(Expr&& expr, Params&& params)
  {   
    // 2-qubit U3 ansatz         
    auto U3_0 = U3<0>(sel<0>(init()), params[0], params[1], params[2]);
    auto U3_1 = U3<1>(sel<1>(all(U3_0)), params[3], params[4], params[5]);

    auto CNOT1 = cnot(sel<1>(), sel<0>(all(U3_1)));

    auto U3_2 = U3<2>(sel<0>(all(CNOT1)), params[6], params[7], params[8]);
    auto U3_3 = U3<3>(sel<1>(all(U3_2)), params[9], params[10], params[11]);

    auto CNOT2 = cnot(sel<0>(), sel<1>(all(U3_3)));

    auto U3_4 = U3<4>(sel<0>(all(CNOT2)), params[12], params[13], params[14]);
    auto U3_5 = U3<5>(sel<1>(all(U3_4)), params[15], params[16], params[17]);

    auto CNOT3 = cnot(sel<1>(), sel<0>(all(U3_5)));

    auto U3_6 = U3<6>(sel<0>(all(CNOT3)), params[18], params[19], params[20]);
    auto U3_7 = U3<7>(sel<1>(all(U3_6)), params[21], params[22], params[23]);

    return all(U3_7);
  }

  // Convert histogram output to expectation value
  template<typename HistType>
  double histToExp(HistType hist)
  {
    // Initialize expectation value an total number of shots to zero
    long expectation = 0;
    long shots = 0;
      
    //Loop over all states
    for(int state = 0; state < hist.size(); state++)
      {
        // Get bitset based on hist index
        std::bitset<2> bits(state);
        
        // Add expectations bases on bitset parity
        if(bits.count() % 2 == 0)
          { // Even
            expectation += (long)hist[state];
          }
        else{
          // Odd
          expectation -= (long)hist[state];
      }
      shots += hist[state];
    }
    // Return expectation over all shots
    return (double)expectation/(double)shots;
  }

  template<QDeviceType DeviceType, typename Params, typename H_coeffs>
  double runVQE(Params&& params, H_coeffs h_coeffs, std::size_t shots)
  {     
    // Generate ansatz (without measurements)
    auto anz = genAnsatz(init(), params);

    // Initalize two quantum devices, each for a measurement basis
    QDevice<DeviceType, 2> qpu0, qpu1;

    // Load ansatz onto QPU and execute
    qpu0(measure(h(sel<0>(anz))));                                   // Measure qubit 0 in the X basis
    auto result0 = qpu0.eval(shots);                                 // Execute for shots times
    auto hist0 = qpu0.template get<QResultType::histogram>(result0); // Retrieve histogram from results
    auto expectation0 = histToExp(hist0);                            // Get expectation value from histogram

    qpu1(measure(h(sel<1>(anz))));                                   // Measure qubit 1 in the X basis
    auto result1 = qpu1.eval(shots);                                 // Execute for shots times
    auto hist1 = qpu1.template get<QResultType::histogram>(result1); // Retrieve histogram from results
    auto expectation1 = histToExp(hist1);                            // Get expectation value from histogram

    // Return expectation value for each Hamiltonian term, scaled by its coeffient
    return h_coeffs.c[0]*expectation0 + h_coeffs.c[1]*expectation1;
  }
};

### The NLopt callback function
Furthermore, we need to implement a function that can be passed to the [NLopt](https://nlopt.readthedocs.io/en/latest/) optimizer. Since it is used as callback function it must match the exact interface requirements of NLopt.

In [ ]:
static double vqa_opt(const std::vector<double> &params, std::vector<double> &grad, void *func_data)
{
  static std::size_t iter=0;
  
  // Create Hamiltonian
  HamiltonianCoeffs H_coeffs;

  // Create VQE struct
  VQE vqe;

  // Run VQE and get expectation function
  double expectation = vqe.runVQE<QDeviceType::cirq_simulator>(params, H_coeffs, 4096);

  // Iteration message:
  std::cout << "Iteration " << std::to_string(iter++)
            << " Expectation value " << std::to_string(expectation) << std::endl;

  return expectation;
}

### Classical validation function
In order to verify the correctness of the quantum computation we also implement a classical validation function that computes the eigenvalues of our Hamiltonian and returns the smallest one. For this purpose we utilize **LibKet**'s built-in linear algebra library [Armadillo](http://arma.sourceforge.net).

In [ ]:
double classicalExpectation()
{
  // Complex contants (0, i, 1)
  std::complex<double> c_0(0,0);
  std::complex<double> c_i(0,1);
  std::complex<double> c_1(1,0);

  HamiltonianCoeffs H_coeffs;

  // Pauli matrices
  arma::cx_mat I = {{c_1,  c_0}, {c_0,  c_1}};
  arma::cx_mat X = {{c_0,  c_1}, {c_1,  c_0}};
  arma::cx_mat H_BK = H_coeffs.c[0]*arma::kron(X, I) + H_coeffs.c[1]*arma::kron(I, X);
  arma::vec eigval = arma::sort(arma::eig_sym(real(H_BK)));

  return eigval[0];
}

### The main program
Last but not least we implement the main program.

**!!! Note that the code block below will not run in this C++ interpreter. We told you that things can go wrong and this is where it happens !!!**

We will therefore switch to the **Terminal session** and open the file `~/LibLet/examples/tutorial17_vqe.cpp`.

In [ ]:
// Set U3 parameters   
std::size_t numParams = 8*3;

// Set NLopt variables     
double _minf;
std::vector<double> _opt_params;

// NLopt optimizer initialization    
std::vector<double> lb;
std::vector<double> ub;
std::vector<double> params;
double tol = 1e-3;
   
// Fill in parameters based on algorithm   
for(int i=0; i < numParams; i++)
{
    lb.push_back(0.0);      // Set lower bounds to 0.0
    ub.push_back(1.0);      // Set upper bounds to 1.0
    params.push_back(0.5);  // Set initial pararms to 0.5
} 

struct OptData{} optData;

// Set parameters
nlopt::opt opt(nlopt::LN_COBYLA, numParams);
opt.set_lower_bounds(lb);
opt.set_upper_bounds(ub);
opt.set_min_objective(vqa_opt, &optData); 
opt.set_xtol_rel(tol);

try
{
    nlopt::result result = opt.optimize(params, _minf);
    _opt_params = params;
}
catch(std::exception &e)
{
    std::cout << "NLopt failed: " << e.what() << std::endl;
}
  
std::cout << "VQE Expectation: " << _minf << std::endl;
std::cout << "Exact Expectation: " << classicalExpectation() << std::endl;